In [0]:
# 重启
# !kill -9 -1
# 安装包
!pip install deepctr

In [0]:
# 挂载谷歌硬盘 从drive导入数据
from google.colab import drive
drive.mount('/content/drive')
# 创建数据文件夹
!mkdir /content/dataset
# 复制文件到本地
# !cp -r /content/drive/My\ Drive/Colab_Notebooks/RS6/L11/data_format1_small /content/dataset/data_format1_small
!cp /content/drive/My\ Drive/Dataset/data_format2.zip /content/dataset/data_format2.zip
!cp /content/drive/My\ Drive/Dataset/data_format1.zip /content/dataset/data_format1.zip
# 切换当前目录
%cd /content/dataset/
# 解压缩文件
!unzip data_format2.zip
!unzip data_format1.zip
# !unzip ./data_format1_small/data_format1_small.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/dataset
Archive:  data_format2.zip
   creating: data_format2/
  inflating: data_format2/test_format2.csv  
  inflating: data_format2/train_format2.csv  
Archive:  data_format1.zip
   creating: data_format1/
  inflating: data_format1/test_format1.csv  
  inflating: data_format1/train_format1.csv  
  inflating: data_format1/user_info_format1.csv  
  inflating: data_format1/user_log_format1.csv  


### 加载数据

In [0]:
# 导入包
import pandas as pd
import gc

In [0]:
# 加载小样本
user_log = pd.read_csv('./data_format1_small/sample_user_log.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('./data_format1_small/sample_user_info.csv')
train_data1 = pd.read_csv('./data_format1_small/train.csv')
submission = pd.read_csv('./data_format1_small/test.csv')
train_data = pd.read_csv('./data_format2/train_format2.csv')

In [0]:
# 加载全量样本
user_log = pd.read_csv('./data_format1/user_log_format1.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('./data_format1/user_info_format1.csv')
train_data1 = pd.read_csv('./data_format1/train_format1.csv')
submission = pd.read_csv('./data_format1/test_format1.csv')
train_data = pd.read_csv('./data_format2/train_format2.csv')

### 数据预处理

In [0]:
train_data1['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)
matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left')
# 使用merchant_id（原列名seller_id）
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0.0,train,6.0,0.0
1,34176,121,0.0,train,6.0,0.0
2,34176,4356,1.0,train,6.0,0.0
3,34176,2217,0.0,train,6.0,0.0
4,230784,4818,0.0,train,0.0,0.0
...,...,...,...,...,...,...
522336,228479,3111,NaN,test,6.0,0.0
522337,97919,2341,NaN,test,8.0,1.0
522338,97919,3971,NaN,test,8.0,1.0
522339,32639,3536,NaN,test,0.0,0.0


In [0]:
# 格式化
user_log['user_id'] = user_log['user_id'].astype('int32')
user_log['merchant_id'] = user_log['merchant_id'].astype('int32')
user_log['item_id'] = user_log['item_id'].astype('int32')
user_log['cat_id'] = user_log['cat_id'].astype('int32')
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int32')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
# 1 for <18; 2 for [18,24]; 3 for [25,29]; 4 for [30,34]; 5 for [35,39]; 6 for [40,49]; 7 and 8 for >= 50; 0 and NULL for unknown
matrix['age_range'].fillna(0, inplace=True)
# 0:female, 1:male, 2:unknown
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')
del user_info, train_data1
gc.collect()
matrix

,user_id,merchant_id,label,origin,age_range,gender
0,34176,3906,0.0,train,6,0
1,34176,121,0.0,train,6,0
2,34176,4356,1.0,train,6,0
3,34176,2217,0.0,train,6,0
4,230784,4818,0.0,train,0,0
...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0
522337,97919,2341,nan,test,8,1
522338,97919,3971,nan,test,8,1
522339,32639,3536,nan,test,0,0


### 特征处理

In [0]:
# User特征处理
groups = user_log.groupby(['user_id'])
# 用户交互行为数量 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
# 使用agg 基于列的聚合操作，统计唯一值的个数 item_id, cat_id, merchant_id, brand_id
#temp = groups['item_id', 'cat_id', 'merchant_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'u2', 'cat_id':'u3', 'merchant_id':'u4', 'brand_id':'u5'})
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')

# 时间间隔特征 u6 按照小时
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10
0,34176,3906,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
1,34176,121,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
2,34176,4356,1.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
3,34176,2217,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
4,230784,4818,0.0,train,0,0,54,31,17,20,19,5.166667,47.0,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,2004,1173,71,278,282,6.000000,1770.0,NaN,26.0,208.0
522337,97919,2341,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0
522338,97919,3971,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0
522339,32639,3536,nan,test,0,0,72,46,24,33,35,5.800000,62.0,1.0,8.0,1.0


In [0]:
# 商家特征处理
groups = user_log.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
matrix

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10,m1_x,m2_x,m3_x,m4_x,m5_x,m6_x,m7_x,m8_x,m9_x,m1_y,m2_y,m3_y,m4_y,m5_y,m6_y,m7_y,m8_y,m9_y,m10
0,34176,3906,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0,16269,5819,308,20,2,14870.0,28.0,410.0,961.0,16269,5819,308,20,2,14870.0,28.0,410.0,961.0,2861
1,34176,121,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0,79865,10931,1179,26,2,72265.0,121.0,4780.0,2699.0,79865,10931,1179,26,2,72265.0,121.0,4780.0,2699.0,4530
2,34176,4356,1.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0,7269,2281,67,15,2,6094.0,16.0,963.0,196.0,7269,2281,67,15,2,6094.0,16.0,963.0,196.0,1088
3,34176,2217,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0,60202,16870,377,5,2,52230.0,101.0,3721.0,4150.0,60202,16870,377,5,2,52230.0,101.0,3721.0,4150.0,7268
4,230784,4818,0.0,train,0,0,54,31,17,20,19,5.166667,47.0,NaN,7.0,NaN,48089,7500,461,27,2,43268.0,129.0,2733.0,1959.0,48089,7500,461,27,2,43268.0,129.0,2733.0,1959.0,3102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,2004,1173,71,278,282,6.000000,1770.0,NaN,26.0,208.0,10105,4154,542,50,18,8997.0,9.0,687.0,412.0,10105,4154,542,50,18,8997.0,9.0,687.0,412.0,1982
522337,97919,2341,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0,5543,1592,352,93,19,4548.0,6.0,815.0,174.0,5543,1592,352,93,19,4548.0,6.0,815.0,174.0,703
522338,97919,3971,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0,28892,7587,272,7,2,24602.0,94.0,2608.0,1588.0,28892,7587,272,7,2,24602.0,94.0,2608.0,1588.0,3050
522339,32639,3536,nan,test,0,0,72,46,24,33,35,5.800000,62.0,1.0,8.0,1.0,14027,4956,322,19,3,12807.0,29.0,793.0,398.0,14027,4956,322,19,3,12807.0,29.0,793.0,398.0,2177


In [0]:
# 按照user_id, merchant_id分组
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) #统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔
matrix

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10,m1_x,m2_x,m3_x,m4_x,m5_x,m6_x,m7_x,m8_x,m9_x,m1_y,m2_y,m3_y,m4_y,m5_y,m6_y,m7_y,m8_y,m9_y,m10,um1,um2,um3,um4,um5,um6,um7,um8,um9
0,34176,3906,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0,16269,5819,308,20,2,14870.0,28.0,410.0,961.0,16269,5819,308,20,2,14870.0,28.0,410.0,961.0,2861,39,20,6,1,36.0,NaN,1.0,2.0,0.850000
1,34176,121,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0,79865,10931,1179,26,2,72265.0,121.0,4780.0,2699.0,79865,10931,1179,26,2,72265.0,121.0,4780.0,2699.0,4530,14,1,1,1,13.0,NaN,1.0,NaN,0.050000
2,34176,4356,1.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0,7269,2281,67,15,2,6094.0,16.0,963.0,196.0,7269,2281,67,15,2,6094.0,16.0,963.0,196.0,1088,18,2,1,1,12.0,NaN,6.0,NaN,0.016667
3,34176,2217,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0,60202,16870,377,5,2,52230.0,101.0,3721.0,4150.0,60202,16870,377,5,2,52230.0,101.0,3721.0,4150.0,7268,2,1,1,1,1.0,NaN,1.0,NaN,0.000000
4,230784,4818,0.0,train,0,0,54,31,17,20,19,5.166667,47.0,NaN,7.0,NaN,48089,7500,461,27,2,43268.0,129.0,2733.0,1959.0,48089,7500,461,27,2,43268.0,129.0,2733.0,1959.0,3102,8,1,1,1,7.0,NaN,1.0,NaN,0.050000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,2004,1173,71,278,282,6.000000,1770.0,NaN,26.0,208.0,10105,4154,542,50,18,8997.0,9.0,687.0,412.0,10105,4154,542,50,18,8997.0,9.0,687.0,412.0,1982,5,2,1,1,4.0,NaN,1.0,NaN,0.016667
522337,97919,2341,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0,5543,1592,352,93,19,4548.0,6.0,815.0,174.0,5543,1592,352,93,19,4548.0,6.0,815.0,174.0,703,2,1,1,1,1.0,NaN,1.0,NaN,0.000000
522338,97919,3971,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0,28892,7587,272,7,2,24602.0,94.0,2608.0,1588.0,28892,7587,272,7,2,24602.0,94.0,2608.0,1588.0,3050,16,5,2,1,12.0,NaN,4.0,NaN,0.150000
522339,32639,3536,nan,test,0,0,72,46,24,33,35,5.800000,62.0,1.0,8.0,1.0,14027,4956,322,19,3,12807.0,29.0,793.0,398.0,14027,4956,322,19,3,12807.0,29.0,793.0,398.0,2177,3,2,1,1,2.0,NaN,1.0,NaN,0.000000


In [0]:
#用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比 不需要这个特征值，得分0.68
# matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
matrix

,user_id,merchant_id,label,origin,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10,m1_x,m2_x,m3_x,m4_x,m5_x,m6_x,m7_x,m8_x,m9_x,m1_y,m2_y,m3_y,m4_y,m5_y,m6_y,m7_y,m8_y,m9_y,m10,um1,um2,um3,um4,um5,um6,um7,um8,um9,r1,r3,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,g_0,g_1,g_2
0,34176,3906,0.0,train,451,256,45,109,108,5.833333,410.0,0.0,34.0,7.0,16269,5819,308,20,2,14870.0,28.0,410.0,961.0,16269,5819,308,20,2,14870.0,28.0,410.0,961.0,2861,39,20,6,1,36.0,0.0,1.0,2.0,0.850000,0.082927,0.027778,0,0,0,0,0,0,1,0,0,1,0,0
1,34176,121,0.0,train,451,256,45,109,108,5.833333,410.0,0.0,34.0,7.0,79865,10931,1179,26,2,72265.0,121.0,4780.0,2699.0,79865,10931,1179,26,2,72265.0,121.0,4780.0,2699.0,4530,14,1,1,1,13.0,0.0,1.0,0.0,0.050000,0.082927,0.076923,0,0,0,0,0,0,1,0,0,1,0,0
2,34176,4356,1.0,train,451,256,45,109,108,5.833333,410.0,0.0,34.0,7.0,7269,2281,67,15,2,6094.0,16.0,963.0,196.0,7269,2281,67,15,2,6094.0,16.0,963.0,196.0,1088,18,2,1,1,12.0,0.0,6.0,0.0,0.016667,0.082927,0.500000,0,0,0,0,0,0,1,0,0,1,0,0
3,34176,2217,0.0,train,451,256,45,109,108,5.833333,410.0,0.0,34.0,7.0,60202,16870,377,5,2,52230.0,101.0,3721.0,4150.0,60202,16870,377,5,2,52230.0,101.0,3721.0,4150.0,7268,2,1,1,1,1.0,0.0,1.0,0.0,0.000000,0.082927,1.000000,0,0,0,0,0,0,1,0,0,1,0,0
4,230784,4818,0.0,train,54,31,17,20,19,5.166667,47.0,0.0,7.0,0.0,48089,7500,461,27,2,43268.0,129.0,2733.0,1959.0,48089,7500,461,27,2,43268.0,129.0,2733.0,1959.0,3102,8,1,1,1,7.0,0.0,1.0,0.0,0.050000,0.148936,0.142857,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,2004,1173,71,278,282,6.000000,1770.0,0.0,26.0,208.0,10105,4154,542,50,18,8997.0,9.0,687.0,412.0,10105,4154,542,50,18,8997.0,9.0,687.0,412.0,1982,5,2,1,1,4.0,0.0,1.0,0.0,0.016667,0.014689,0.250000,0,0,0,0,0,0,1,0,0,1,0,0
522337,97919,2341,nan,test,55,29,14,17,17,4.750000,46.0,0.0,8.0,1.0,5543,1592,352,93,19,4548.0,6.0,815.0,174.0,5543,1592,352,93,19,4548.0,6.0,815.0,174.0,703,2,1,1,1,1.0,0.0,1.0,0.0,0.000000,0.173913,1.000000,0,0,0,0,0,0,0,0,1,0,1,0
522338,97919,3971,nan,test,55,29,14,17,17,4.750000,46.0,0.0,8.0,1.0,28892,7587,272,7,2,24602.0,94.0,2608.0,1588.0,28892,7587,272,7,2,24602.0,94.0,2608.0,1588.0,3050,16,5,2,1,12.0,0.0,4.0,0.0,0.150000,0.173913,0.333333,0,0,0,0,0,0,0,0,1,0,1,0
522339,32639,3536,nan,test,72,46,24,33,35,5.800000,62.0,1.0,8.0,1.0,14027,4956,322,19,3,12807.0,29.0,793.0,398.0,14027,4956,322,19,3,12807.0,29.0,793.0,398.0,2177,3,2,1,1,2.0,0.0,1.0,0.0,0.000000,0.129032,0.500000,1,0,0,0,0,0,0,0,0,1,0,0


In [0]:
# 创建用户行为sequence
lbe_action_type={0:1,1:2,2:3,3:4}
user_log['action_type']=user_log['action_type'].map(lbe_action_type)
# 把user_log里同user的这些数据合并成一个list
temp=pd.DataFrame(user_log.groupby('user_id')['merchant_id','action_type'].agg(lambda x:list(x)))
# 列名称改成hist_merchant_id 和 hist_action_type 
temp.columns=['hist_merchant_id','hist_action_type']
# print(temp)
# 统计时间间隔
matrix = matrix.merge(temp, on=['user_id'], how='left')
matrix

In [0]:
# 截取，补缺到定长M个
M=500
for feature in ['hist_merchant_id','hist_action_type']:
    matrix[feature]=matrix[feature].map(lambda x:np.array(x+[0]*(M-len(x)))[:M])

### 训练集和测试集划分

In [0]:
# 分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
del temp, matrix
gc.collect()

1555

### 模型训练（使用NN模型）

In [0]:
# 导入用到的模型包
from deepctr.inputs import SparseFeat,VarLenSparseFeat,DenseFeat,get_feature_names
from deepctr.models import DIN, DIEN, DSIN

In [0]:
train_X['action_type']=3
feature_columns = []
for column in train_X.columns:
  if column != 'hist_merchant_id' and column != 'hist_action_type':
    print(column)
    num = train_X[column].nunique()
    if num > 10000:
        dim = 10
    else:
        if num > 1000:
            dim = 8
        else:
            dim = 4
    print(num)
    if column  == 'user_id':
        feature_columns += [SparseFeat(column, 19111+1, embedding_dim=dim)]
    elif column  == 'merchant_id':
        feature_columns += [SparseFeat(column, 4994+1, embedding_dim=dim)]
    elif column  == 'action_type':
        feature_columns += [SparseFeat(column, 4+1, embedding_dim=dim)]
    else:
        feature_columns += [DenseFeat(column, 1)]

In [0]:
print('M=', M)

# maxlen为历史信息的长度，vocabulary_size为onehot的长度
feature_columns += [VarLenSparseFeat('hist_merchant_id', maxlen=M, vocabulary_size=19111+1, embedding_dim=8, embedding_name='merchant_id'),
                   VarLenSparseFeat('hist_action_type', maxlen=M, vocabulary_size=4+1, embedding_dim=4, embedding_name='action_type')]
hist_features=['merchant_id','action_type']
print(feature_columns)

# 使用DIN模型
model=DIN(feature_columns, hist_features)
# 使用Adam优化器，二分类的交叉熵
model.compile('adam', 'binary_crossentropy', metrics=['binary_crossentropy'])

# 组装train_model_input，得到feature names，将train_X转换为字典格式
feature_names=list(train_X.columns)
train_model_input = {name:train_X[name].values for name in feature_names}
# histroy输入必须是二维数组
from tqdm import tqdm
for fea in ['hist_merchant_id','hist_action_type']:
    l = []
    for i in tqdm(train_model_input[fea]):
        l.append(i)
    train_model_input[fea]=np.array(l)
history = model.fit(train_model_input, train_y, verbose=True, epochs=10, validation_split=0.2,batch_size=512)


### 测试集预测

In [0]:
# 转换test__model_input
test_data['action_type']=3
test_model_input = {name:test_data[name].values for name in feature_names}
from tqdm import tqdm
for fea in ['hist_merchant_id','hist_action_type']:
    l = []
    for i in tqdm(test_model_input[fea]):
        l.append(i)
    test_model_input[fea]=np.array(l)

# 得到预测结果
prob = model.predict(test_model_input)
submission['prob'] = prob
submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('prediction.csv', index=False)